# What is function calling?
- Why use function calling?
- Use cases for function calling
    - Going from unstructured to consistent structured output
    - Calling multiple functions to provide complex responses
    - Calling external APIs

In [6]:
from openai import OpenAI
import json
client = OpenAI()

In [2]:
response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[{"role": "user", "content": "Please write down four trees with their scientific names in json."} ],
    response_format={"type": "json_object"})

print(response.choices[0].message.content)

{
  "trees": [
    {
      "common_name": "Oak",
      "scientific_name": "Quercus"
    },
    {
      "common_name": "Maple",
      "scientific_name": "Acer"
    },
    {
      "common_name": "Pine",
      "scientific_name": "Pinus"
    },
    {
      "common_name": "Birch",
      "scientific_name": "Betula"
    }
  ]
}


## Implementing function calling
> from openai import OpenAI  
> client = OpenAI(api_key="ENTER YOUR KEY HERE")  
> response= client.chat.completions.create(  
> &nbsp;&nbsp;&nbsp;&nbsp;model="gpt-4o-mini",  
> &nbsp;&nbsp;&nbsp;&nbsp;messages=messages,  
> &nbsp;&nbsp;&nbsp;&nbsp;tools=function_definition)

Setting up function calling

function_definition = [{'type': 'function','function': {'name': 'extract_job_info','description': 'Get the job information from the body of the input text','parameters': {'type': 'object','properties': 'job': {'type': 'string','description': 'Job title'},'location': {'type': 'string','description': 'Office location'},              ...           }    }]



In [12]:
from openai import OpenAI
# client = OpenAI(api_key="ENTER YOUR KEY HERE")

function_definition = [{
    'type': 'function',
    'function': {
        'name': 'extract_job_info',
        'description': 'Get the job information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'job': {
                    'type': 'string', 
                    'description': 'Job title'
                },
                'location': {
                    'type': 'string', 'description': 'Location'
                }
            }
        }
    }
}]

job_description = """
    "We are currently seeking a highly skilledData Scientist to join our innovative team atthe company's headquarters in San Francisco,CA. In this role, you will have the opportunityto work on complex data analysis andmodeling projects that drive our strategicdecisions. Requirements: Minimum 3 years ofexperience in data science with Python andAWS, Azure or GCP.
"""

messages = [
    {"role": "system", "content": "You are a helpful assistant that extracts job information from text and timezones for the location of the job."},
    {"role": "user", "content": job_description}
]

response= client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages, 
    tools=function_definition)

print(response.choices[0].message.tool_calls[0].function)  # extract message

Function(arguments='{"job": "Data Scientist", "location": "San Francisco, CA"}', name='extract_job_info')


In [13]:
# Append a new function definition to the list
function_definition.append(
    {
        'type': 'function',
        'function': {
            'name': 'get_timezone',
            'description': 'Return the timezone corresponding to the location in the job advert',
            'parameters': {
                'type': 'object',
                'properties': {
                    'timezone': {
                        'type': 'string',
                        'description': 'Timezone'
                    }
                }
            }
        }
    })

response= client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages, 
    tools=function_definition)
print(response.choices[0].message.tool_calls[1].function)  # extract message

Function(arguments='{"timezone": "San Francisco, CA"}', name='get_timezone')


# Setting specific functions
- You can set `tool_choice`
    - auto
    - a specific function
    

In [14]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=function_definition,
    tool_choice='auto')

print(response.choices[0].message.tool_calls)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=function_definition,
    tool_choice={'type': 'function', 'function': {'name': 'extract_job_info'} }
)

print(response.choices[0].message.tool_calls)


[ChatCompletionMessageToolCall(id='call_CtH1BApXQXsSEJ6ZjVu4l1oO', function=Function(arguments='{"job": "Data Scientist", "location": "San Francisco, CA"}', name='extract_job_info'), type='function'), ChatCompletionMessageToolCall(id='call_hHf69pavxF4eyCKfTyjyLXRx', function=Function(arguments='{"timezone": "San Francisco, CA"}', name='get_timezone'), type='function')]
[ChatCompletionMessageToolCall(id='call_vv7d2JgwlQeGGmrMsgoF03GH', function=Function(arguments='{"job":"Data Scientist","location":"San Francisco, CA"}', name='extract_job_info'), type='function')]


# Use Pydantic to simplify creating function definition

In [18]:
from pydantic import BaseModel, Field

class ExtractJobInfoParams(BaseModel):
    job: str = Field(..., description="Job title")
    location: str = Field(..., description="Location")

class GetTimezoneParams(BaseModel):
    timezone: str = Field(..., description="Timezone")

function_definition = [
    {
        'type': 'function',
        'function': {
            'name': 'extract_job_info',
            'description': 'Get the job information from the body of the input text',
            'parameters': ExtractJobInfoParams.schema()
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'get_timezone',
            'description': 'Return the timezone corresponding to the location in the job advert',
            'parameters': GetTimezoneParams.schema()
        }
    }
]

C:\Users\dtrun\AppData\Local\Temp\ipykernel_24720\2262845222.py:16: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  'parameters': ExtractJobInfoParams.schema()
C:\Users\dtrun\AppData\Local\Temp\ipykernel_24720\2262845222.py:24: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  'parameters': GetTimezoneParams.schema()


In [19]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=function_definition,    
)

print(response.choices[0].message.tool_calls)

[ChatCompletionMessageToolCall(id='call_TN84as00MnAd4CtXFH1iOirN', function=Function(arguments='{"job": "Data Scientist", "location": "San Francisco, CA"}', name='extract_job_info'), type='function'), ChatCompletionMessageToolCall(id='call_RzVimpfvccHXcFLOGgN1pXhC', function=Function(arguments='{"timezone": "San Francisco, CA"}', name='get_timezone'), type='function')]


# Calling external APIs
- Use `requests` library to call external API


In [21]:
# Python requests library for APIs
import requests
url = "https://api.artic.edu/api/v1/artworks/search"
querystring = { "q": "painting" }
response = requests.request("GET", url, params=querystring)
if response.status_code == 200:
    data = response.json()
    print(json.dumps(data, indent=2))

{
  "preference": null,
  "pagination": {
    "total": 96829,
    "limit": 10,
    "offset": 0,
    "total_pages": 9683,
    "current_page": 1
  },
  "data": [
    {
      "_score": 101.7596,
      "id": 82410,
      "api_model": "artworks",
      "api_link": "https://api.artic.edu/api/v1/artworks/82410",
      "is_boosted": false,
      "title": "Untitled (Painting)",
      "thumbnail": {
        "lqip": "",
        "width": 10184,
        "height": 9067,
        "alt_text": "A rectangular painting on light, vivid orange features a deeper orange block set atop a yellow block. The colors of each element are only loosely blended and lightly applied, so various tones are visible."
      },
      "timestamp": "2025-07-22T23:22:34-05:00"
    },
    {
      "_score": 97.03791,
      "id": 102581,
      "api_model": "artworks",
      "api_link": "https://api.artic.edu/api/v1/artworks/102581",
      "is_boosted": false,
      "title": "Painting",
      "thumbnail": {
        "lqip": "",
     

In [30]:
def get_artwork(keyword):
    url = "https://api.artic.edu/api/v1/artworks/search"    
    querystring = {"q":keyword}    
    response = requests.request("GET", url, params=querystring)
    return response.text


In [26]:
messages=[
    {"role": "system", "content": "You are an AI assistant, a specialist in history of art. You should interpret the user prompt, and based on it extract one     keyword for recommending artwork related to their preference."},
    {"role": "user", "content": "I don't have much time to visit the museum and would     like some recommendations. I like the seaside and quiet places."}
]

In [27]:
# Declare the function definition
class ArtworkParams(BaseModel):
    keyword: str = Field(..., description="Keyword for artwork search")

function_definition = [
    {
        'type': 'function',
        'function': {
            'name': 'get_artwork',
            'description': 'Get artwork based on a keyword',
            'parameters': ArtworkParams.schema()
        }
    }
]

C:\Users\dtrun\AppData\Local\Temp\ipykernel_24720\329715027.py:11: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  'parameters': ArtworkParams.schema()


In [28]:
response = client.chat.completions.create(
    model="gpt-4o-mini", 
    messages=messages,
    tools=function_definition)

print(response.choices[0].message.tool_calls[0].function)  # extract message


Function(arguments='{"keyword":"seaside"}', name='get_artwork')


## Bringing it all together
- Get the parsed content of the function calling to call external API


In [29]:
import json

if response.choices[0].finish_reason=='tool_calls':
    function_call = response.choices[0].message.tool_calls[0].function
    if function_call.name == "get_artwork":
        artwork_keyword = json.loads(function_call.arguments)["keyword"]
        artwork = get_artwork(artwork_keyword)
        if artwork: 
            print(f"Here are some recommendations: {[i['title'] for i in json.loads(artwork)['data']]}")
        else:
            print("Apologies, I couldn't make any recommendations based on the request.")
    else:
        print("Apologies, I couldn't find any artwork.")
else:
    print("I am sorry, but I could not understand your request.")

Here are some recommendations: ['Seaside, Port of Honfleur', 'Little Landscape at the Seaside (Kleine Landschaft am Meer)', 'Museum of Contemporary Art, Niterói, Rio de Janeiro, Brazil, Four Sketches', 'A seaside outing', 'Waitress at a Seaside Teahouse', 'Summer Landscape with Letter Forms', 'Seaside Excursions Southend', 'US Post Office, Seaside, Florida, from the series "Sweet Earth: Experimental Utopias in America"', 'Rustic Seaside Scene', 'A Holiday']
